In [52]:
import pandas as pd
import pandas_utils as pu

In [53]:
libraries = {
  "iocb_peptide": r"C:\git\msn_library\data\iocb_libraries\iocb_peptide_library_cleaned.tsv",
  "nih": r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv",
  "mce_nencka": r"C:\git\msn_library\data\iocb_libraries\Radim_mce_complete_cleaned.tsv",
  "mce": r"C:\git\msn_library\data\library\mce_library_all_cleaned.tsv",
  
}

In [54]:
dfs = []
for key, value in libraries.items():
  df = pu.read_dataframe(value).copy()
  df = df[df["monoisotopic_mass"] > 114]
  df = df.drop_duplicates(["inchikey", "unique_sample_id"])
  df["library"] = key
  # if key in acquired:
  #   df["acquired"] = True
  # if key in collaborators:
  #   df["collaborators"] = True
  dfs.append(df)


merged_df = pd.concat(dfs, ignore_index=True)

C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (59,97,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")
C:\git\msn_library\pandas_utils.py:34: DtypeWarning: Columns (156) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")


In [55]:
merged_df

,CLogP,Clinical Information,Clinical Information_clinical_phase,ID,IDNUMBER,PSA,Saltdata,Source1536wWell,any_phase,availability,...,Information,Pathway,Plate Location,RackCode,library_id,mixed_location_plate1,mixed_location_plate2,mixed_location_plate3,original_location,target
0,-1.0,NaN,0,1.0,P1086813,69.6,NaN,A1,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.5,NaN,0,89.0,P0120630213,71.2,NaN,A2,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.6,NaN,0,177.0,P7020554651,84.2,NaN,A3,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.6,NaN,0,265.0,P1305501,54.1,NaN,A4,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.2,NaN,0,9.0,P3644480,87.1,NaN,A5,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20594,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,False,-1.0,...,NaN,NaN,NaN,HYCPK16573,pluskal_mce,1D3_L18,2D3_L18,3D3_L18,B2,Others
20595,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,HYCPK16573,pluskal_mce,1D3_L19,2D3_L19,3D3_L19,B3,endogenous metabolite; FGFR; Wnt
20596,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,False,NaN,...,NaN,NaN,NaN,HYCPK16573,pluskal_mce,1D3_L19,2D3_L19,3D3_L19,B5,Others
20597,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,True,NaN,...,NaN,NaN,NaN,HYCPK16573,pluskal_mce,1D3_L19,2D3_L19,3D3_L19,B6,Autophagy


In [61]:
sub = merged_df.groupby("library")

In [73]:
def extract_row(libid: str, df:pd.DataFrame) -> dict:
  unique_df = df.drop_duplicates(["inchikey"])
  return {
    "library": libid,
    "total_compounds": len(df),
    "unique_structures": len(unique_df),
    "natural_product": len(unique_df[unique_df["natural_product"] == 1]),
    "no_natural_product": len(unique_df[unique_df["natural_product"] == 0]),
    "unknown": len(unique_df[(unique_df["natural_product"] != 0) & (unique_df["natural_product"] != 1) ]),
    "lotus": len(unique_df[unique_df["lotus_ncbi_id"].notnull()]),
    "neutral": len(unique_df[unique_df["molecular_species"] == "NEUTRAL"]),
    "acid": len(unique_df[unique_df["molecular_species"] == "ACID"]),
    "base": len(unique_df[unique_df["molecular_species"] == "BASE"]),
    "zwitterion": len(unique_df[unique_df["molecular_species"] == "ZWITTERION"]),
    "any_clinical_phase": len(unique_df[unique_df["any_phase"] == True]),
    "clinic": len(unique_df[unique_df["clinical_phase"] == 4]),
  }

lib_rows = []
for group in sub:
  libid = group[0]
  groupdf = group[1]

  row = extract_row(libid, groupdf)
  lib_rows.append(row)
  
# get summary stats
row = extract_row("Summary", merged_df)
lib_rows.append(row)
lib_rows
statistic_df = pd.DataFrame(lib_rows)

In [74]:
statistic_df

,library,total_compounds,unique_structures,natural_product,no_natural_product,unknown,lotus,neutral,acid,base,zwitterion,any_clinical_phase,clinic
0,iocb_peptide,1298,1297,42,76,1179,2,34,78,2,4,3,0
1,mce,10315,9681,3244,5030,1407,2422,5049,1601,1096,284,5247,2483
2,mce_nencka,4998,4998,43,582,4373,10,530,40,48,2,54,36
3,nih,3988,3915,677,189,3049,2476,659,189,11,3,143,57
4,Summary,20599,19668,3849,5823,9996,4725,6134,1853,1142,291,5341,2517


In [ ]:
# create grid
cols = 4
rows = math.ceil(len(value_columns)/cols)
fig, axs = plt.subplots(rows, cols, sharey=True, figsize=(20, 20))

discrete_values_start_index = 5

for index, column in enumerate(value_columns):
  ci = index % cols
  ri = math.floor(index / cols)
  if index >= discrete_values_start_index:
    sns.histplot(data=df, binwidth=1, x=column, ax=axs[ri, ci])
  else:
    sns.histplot(data=df, x=column, ax=axs[ri, ci])

plt.show()
save_fig("histo_multi")